In [2]:
"""
GOAL OF THIS FILE
This code will ideally be used for the unsupervised training algorithm (in this case, K-means), which generates a bank of filters D
extract 16-by-16 pixel grayscale patches represented as a vector of 256 pixel intensities
I.E. Crop and scale images while keeping RGB channels 

FOR THIS FILE WE ARE ATTEMPTING TO NORMALIZE AND WHITEN OUR PATCHES
Implementing method outlined in this paper:
https://www-cs.stanford.edu/~acoates/papers/coatesng_nntot2012.pdf
"""

import numpy as np
from cv2 import cv2 #Use 'pip install opencv-python' to get module if you don't have it
import glob
import os

In [28]:
class GetImage(object):
    #Load raw image files
    
    #initialize the path (TODO - make basepath an input parameter, don't hardcode too much)

    #Data should be stored in Users/YOUR_USER_NAME/321_galaxies/images_training_rev1
    def __init__(self, username, num_pics=10):

        self.basepath = '/Users/'+username+'/321_galaxies/'
        self.imPath_test = 'images_test_rev1/'
        self.imPath_training =  'images_training_rev1/'
       
        #create variable that stores the image
        self.images = []
        i=0
        for filename in glob.glob(self.basepath + self.imPath_training+'*.jpg'): #assuming jpg
            if(i >= num_pics):
                break
            im=cv2.imread(filename)
            self.images.append(im)
            i +=1 
        self.cropped_images = []
        self.scaled_images = []
        self.patches = []
       

    #Crop image from 424x424 to 160x160. To do we crop axes from {x=0, y=0, x=424, y=424 } => {x=132, y=132, x=292, 400=292}
    def crop(self, pixels_to_keep = 160):
        for galaxyPic in self.images:
            #Cast as int in order to use variables in the slicing indices
            center = int(424/2)
            dim = int(pixels_to_keep/2)
            cropmin = center - dim
            cropmax = center + dim
            self.cropped_images.append(galaxyPic[cropmin:cropmax, cropmin:cropmax])
        return self

    #function that takes an image and scales the entire image to its new size (unlike crop, image stays intact). 
    def scale(self, new_size = 16):
        for galaxyPic in self.cropped_images:
            dimensions = (int(new_size), int(new_size))
            self.scaled_images.append(cv2.resize(galaxyPic, dimensions))
        return self
    
    #function that crops out random 4x4 patches of an image.
    def patch(self, p_size=4, num_patches_per_image = 5):
        for i in range(0, num_patches_per_image):
            for galaxyPic in self.scaled_images:
                patch_x = np.random.randint(2, 14)
                patch_y = np.random.randint(2, 14)
                dim = int(p_size/2)
                patchminx = patch_x - dim
                patchminy = patch_y - dim
                patchmaxx = patch_x + dim
                patchmaxy = patch_y + dim
                self.patches.append(galaxyPic[patchminx:patchmaxx, patchminy:patchmaxy])
        self.patches = np.vstack(self.patches)
        return self

In [93]:
#Testing the class by creating a 'galaxyPic' object, cropping it, scaling it, and saving the new image
galaxyPics = GetImage('samsonmercier/Desktop')
galaxyPics.crop()
galaxyPics.scale()
galaxyPics.patch()

i=0

for patch in galaxyPics.patches:
    cv2.imwrite("patches/patchno"+str(i)+".jpg", patch)
    i += 1
    #cv2.waitKey(0)

In [94]:
## Function that normalizes the pixels. Takes vstack of patches as input
def normalization(patches):
    patches = patches.astype('float32')
    for i in range(len(patches)):
        patch_mean = np.mean(patches[i], axis=(0,1))
        patch_var = np.std(patches[i], axis=(0,1))**2
        patches[i] = (patches[i] - patch_mean) / np.sqrt(patch_var+10)
    return patches

## Function that runs ZCA whitening
def whiten(patches):
    for i in range(len(patches)):
        patch_cov = np.cov(patches[i], rowvar=0)
        patch_mean = np.mean(patches[i], axis=(0,1))
        d, v = np.linalg.eig(patch_cov)
        p = np.dot(v, np.dot(np.diag(np.sqrt(1 / (d + 0.1))), v.T))
        patches[i] = np.dot(patches[i] - patch_mean, p)
    return patches

In [95]:
pixels = normalization(galaxyPics.patches)
new_pixels = whiten(pixels)
pixels.shape

(200, 4, 3)

In [96]:
print(new_pixels[0])

[[-1.3893983  -1.7513521   2.8360186 ]
 [-1.2647171   0.05212893  2.5971808 ]
 [-2.1905177  -0.60810196  2.1198492 ]
 [-2.5344088  -1.2762433   1.1406269 ]]


In [ ]:
#To avoid empty clusters eed to initialize clusters from a Normal distribution 
#and normalize them to unit length?
def Kmeans(L, k = 200, iters = 20, batch_size = 300):
    L2 = np.sum(L**2, 1, keepdims=True)
    #initialize centroids
    centroids = np.random.randn(k, L.shape[1], L.shape[2]) * 0.1
    for iteration in range(1, iters+1):
        c2 = np.sum(centroids**2, 1, keepdims=True)
        summation = np.zeros((k, L.shape[1], L.shape[2]))
        counts = np.zeros((k, 1))
        loss = 0
        for i in range(0, L.shape[0], batch_size):
            last_index = min(i + batch_size, L.shape[0])
            m = last_index - i

            # shape (k, batch_size) - shape (k, 1)
            tmp = np.dot(centroids, L[i:last_index].T) - c2
            # shape (batch_size, )
            indices = np.argmax(tmp, 0)
            # shape (1, batch_size)
            val = np.max(tmp, 0, keepdims=True)

            loss += np.sum((0.5 * L2[i:last_index]) - val.T)

            # Don't use a sparse matrix here
            S = np.zeros((batch_size, k))
            S[range(batch_size), indices] = 1

            # shape (k, n_pixels)
            this_sum = np.dot(S.T, L[i:last_index, :])
            summation += this_sum

            this_counts = np.sum(S, 0, keepdims=True).T
            counts += this_counts 
            
        centroids = summation / counts
        
        bad_indices = np.where(counts == 0)[0]
        centroids[bad_indices, :] = 0

        assert not np.any(np.isnan(centroids))
    return centroids